### Дифференциальное уравнение методом сеток с использованием ряда Фурье.

$\phi (x) = \sin(\pi x)$

In [1]:
import numpy as np
import pandas as pd
from scipy import integrate

In [2]:
# вспомогательные функции
def phi(x):
    return np.sin(np.pi * x)


def psi(x, p):
    return np.sqrt(2) * np.sin(np.pi * x * p)


N = 5
M = 5
h = 1 / N
T = 0.1
tau = T / M

x_arr = np.linspace(0, 1, N + 1)
t_arr = np.linspace(0, T, M + 1)
P = 15

In [3]:
# сетки Фурье
def coefs_Fourier(P):
    coefs = []
    for p in range(1, P):
        coefs.append(integrate.quad(lambda x: phi(x) * psi(x, p), 0, 1)[0])
    return np.around(coefs, 3)


def Fourier_solve(coefs, P, x, t):
    res = 0
    for p in range(P - 1):
        res += coefs[p] * np.exp(-np.pi ** 2 * (p + 1) ** 2 * t) * psi(x, p + 1)
    return res


def coefs_dif_Fourier():
    coefs = []
    for p in range(1, N):
        coef = 0
        for i in range(1, N):
            coef += phi(i * h) * psi(i * h, p)
        coef *= h
        coefs.append(coef)
    return coefs


def dif_Fourier_solve(coefs, x, t):
    res = 0
    for p in range(N - 1):
        res += coefs[p] * np.exp(-np.pi ** 2 * (p + 1) ** 2 * t) * psi(x, p + 1)
    return res

def coefs_grid_Fourier():
    coefs = []
    for p in range(1, N):
        coef = 0
        for i in range(1, N):
            coef += phi(i * h) * psi(i * h, p)
        coef *= h
        coefs.append(coef)
    return coefs

def lambdas(param):
    l = []
    for p in range(1, N):
        tmp1 = 1 - (4 * (1 - param) * tau / h ** 2) * np.sin(p * np.pi * h / 2) ** 2
        tmp2 = 1 + (4 * param * tau / h ** 2) * np.sin(p * np.pi * h / 2) ** 2
        l.append(tmp1 / tmp2)
    return l

def grid_Fourier_solve(coefs, l, x, t):
    res = 0
    k = t / tau
    for p in range(N - 1):
        res += coefs[p] * (l[p] ** k) * psi(x, p + 1)
    return res

In [4]:
# непрерывный ряд Фурье
coeffs = coefs_Fourier(P)
uf = []
for t in t_arr:
    arr = []
    for x in x_arr:
        arr.append(Fourier_solve(coeffs, P, x, t))
    uf.append(arr)

uf_table = pd.DataFrame(data = uf, index=t_arr, columns=x_arr)
uf_table.columns.name = "t \\ x"
uf_table

t \ x,0.0,0.2,0.4,0.6000000000000001,0.8,1.0
0.00,0.0,0.587696,0.950913,0.950913,0.587696,1.224462e-16
0.02,0.0,0.482422,0.780575,0.780575,0.482422,1.005122e-16
0.04,0.0,0.396005,0.640749,0.640749,0.396005,8.250736e-17
0.06,0.0,0.325068,0.525971,0.525971,0.325068,6.772771e-17
0.08,0.0,0.266838,0.431753,0.431753,0.266838,5.559556e-17
0.10,0.0,0.219039,0.354413,0.354413,0.219039,4.563665e-17


In [5]:
# дискретный ряд Фурье
coeffs = coefs_dif_Fourier()
duf = []
for t in t_arr:
    arr = []
    for x in x_arr:
        arr.append(dif_Fourier_solve(coeffs, x, t))
    duf.append(arr)

duf_table = pd.DataFrame(data = duf, index=t_arr, columns=x_arr)
duf_table.columns.name = "t \\ x"
duf_table

t \ x,0.0,0.2,0.4,0.6000000000000001,0.8,1.0
0.00,0.0,0.587785,0.951057,0.951057,0.587785,1.224647e-16
0.02,0.0,0.482495,0.780693,0.780693,0.482495,1.005274e-16
0.04,0.0,0.396065,0.640846,0.640846,0.396065,8.251982e-17
0.06,0.0,0.325117,0.526051,0.526051,0.325117,6.773794e-17
0.08,0.0,0.266878,0.431818,0.431818,0.266878,5.560395e-17
0.10,0.0,0.219072,0.354466,0.354466,0.219072,4.564355e-17


In [6]:
def error_estimation(coefs):
    error_estimation = []
    for t in t_arr:
        arr = []
        for x in x_arr:
            arr.append(Fourier_solve(coefs_Fourier(P), P, x, t) - dif_Fourier_solve(coefs, x, t))
        error_estimation.append(arr)
    error_estimation_table = pd.DataFrame(data = error_estimation, index=t_arr, columns=x_arr)
    error_estimation_table.columns.name = "t \\ x"
    return error_estimation_table  

error_estimation(coefs_dif_Fourier())

t \ x,0.0,0.2,0.4,0.6000000000000001,0.8,1.0
0.00,0.0,-0.000089,-0.000144,-0.000144,-0.000089,-1.849356e-20
0.02,0.0,-0.000073,-0.000118,-0.000118,-0.000073,-1.518079e-20
0.04,0.0,-0.000060,-0.000097,-0.000097,-0.000060,-1.246143e-20
0.06,0.0,-0.000049,-0.000079,-0.000079,-0.000049,-1.022920e-20
0.08,0.0,-0.000040,-0.000065,-0.000065,-0.000040,-8.396831e-21
0.10,0.0,-0.000033,-0.000054,-0.000054,-0.000033,-6.892696e-21


In [7]:
# сеточное решение по дискретному ряду Фурье
def grid_param(param):
    coeffs = coefs_grid_Fourier()
    l = lambdas(param)
    guf = []
    for t in t_arr:
        arr = []
        for x in x_arr:
            arr.append(grid_Fourier_solve(coeffs, l, x, t))
        guf.append(arr)

    grid_table = pd.DataFrame(data = guf, index=t_arr, columns=x_arr)
    grid_table.columns.name = "t \\ x"
    return grid_table

# параметр 0

table = grid_param(0)
table

t \ x,0.0,0.2,0.4,0.6000000000000001,0.8,1.0
0.00,0.0,0.587785,0.951057,0.951057,0.587785,1.224647e-16
0.02,0.0,0.475528,0.769421,0.769421,0.475528,9.907601e-17
0.04,0.0,0.384710,0.622475,0.622475,0.384710,8.015417e-17
0.06,0.0,0.311237,0.503593,0.503593,0.311237,6.484609e-17
0.08,0.0,0.251796,0.407415,0.407415,0.251796,5.246159e-17
0.10,0.0,0.203707,0.329606,0.329606,0.203707,4.244232e-17


In [8]:
# параметр 1/2
table = grid_param(0.5)
table

t \ x,0.0,0.2,0.4,0.6000000000000001,0.8,1.0
0.00,0.0,0.587785,0.951057,0.951057,0.587785,1.224647e-16
0.02,0.0,0.485313,0.785254,0.785254,0.485313,1.011147e-16
0.04,0.0,0.400706,0.648356,0.648356,0.400706,8.348686e-17
0.06,0.0,0.330849,0.535325,0.535325,0.330849,6.893214e-17
0.08,0.0,0.273170,0.441999,0.441999,0.273170,5.691482e-17
0.10,0.0,0.225547,0.364943,0.364943,0.225547,4.699255e-17
